In [19]:
import sys
import os

if '..//..//' not in sys.path:
    sys.path.append('..//../')

from opmsim.optical_system_base import OpticalSystem
from opmsim import optical_elements
from opmsim import dipole_source
import numpy as np
from matplotlib import pyplot as plt


## Element setup and save paths

In [20]:
optical_elements

<module 'opmsim.optical_elements' from 'C:\\Users\\lnr19\\code\\OPMsim\\notebooks\\Simulation setups\\..//..\\opmsim\\optical_elements.py'>

In [39]:
savepath = "C:/Users/lnr19/OneDrive - Imperial College London/thesis/for _thesis/modelling/pupils/"

savepath_full = os.path.join(savepath, "dOPM_edmund2_fin")
if not os.path.exists(savepath_full):
    os.makedirs(savepath_full)
save_filename = os.path.join(savepath_full,'collection_efficiency.txt')
save_details = os.path.join(savepath_full,'system_focal_lengths.txt')


# setup
# O1
M_O1 = 60
NA_O1 = 1.2
n_O1 =1.33
f_TL1 = 0.2  # Nikon
f_O1 = f_TL1/M_O1

# O2
M_O2 = 50
NA_O2 = 0.95
f_TL2 = 0.162 #0.15#0.180  # Olympus
f_O2 = 0.180/M_O2

perfect_mirror = False
retardance = True  # false to ignore retardance effect
film_mat = "SiO"
# film_mat = "perfect mirror"
film_ri = "../../refractive_index_data/"+film_mat+".txt"
film_thickness = 131e-9 # 161e-9 # 131e-9

opm_angles = [0, 35, 45]


In [40]:
dipole_count = 7500

options = {
    'ray_count': 10000,
    'draw_rays': False,
    'max_rays_stored': 10000,
}

plot_options = {
    'title': 'Pupil intensity at final image plane of dOPM microscope',
    'add_sim_details': True,
    'max_r_in': 0,
    'scale_range': None# [0,0.3]
}

#opm_angle = 35*np.pi/180

O1 = optical_elements.SineLens(NA=NA_O1, focal_length=f_O1, n=n_O1)
TL1 = optical_elements.SineLens(NA=0.1, focal_length=f_TL1, n=1)
O2 = optical_elements.SineLens(NA=NA_O2, focal_length=f_O2, n=1)
TL2 = optical_elements.SineLens(NA=0.1, focal_length=f_TL2, n=1)

diag = optical_elements.DiagonalMatrix(-1)

QWP = optical_elements.WavePlate(np.pi/4, np.pi/2)
QWP2 = optical_elements.WavePlate(-np.pi/4, np.pi/2)

LP = optical_elements.LinearPolariser(np.pi/2)
LP2 = optical_elements.LinearPolariser(0)


Ag_mirror = lambda opm_angle: optical_elements.FlatMirror(opm_angle*np.pi/360, film_thickness,\
                                            fresnel_debug_savedir="C:/Users/lnr19/code/OPMsim/output",
                                            n_metal_file = "../../refractive_index_data/Ag.txt",\
                                            n_film_file = film_ri, perfect_mirror=perfect_mirror,\
                                            retardance=retardance)

perf_mirror = lambda opm_angle: optical_elements.FlatMirror(opm_angle*np.pi/360, film_thickness,\
                                            fresnel_debug_savedir="C:/Users/lnr19/code/OPMsim/output",
                                            n_metal_file = "../../refractive_index_data/Ag.txt",\
                                            n_film_file = film_ri, perfect_mirror=True)

elements_fun = lambda opm_angle: [O1, TL1, TL2, LP, QWP, O2, Ag_mirror(opm_angle), O2, QWP, LP2]
elements_s_fun = lambda opm_angle: [O1, TL1, TL2, LP2, QWP, O2, Ag_mirror(opm_angle), O2, QWP, LP]

elements_perf_fun = lambda opm_angle: [O1, TL1, TL2, LP, QWP, O2, perf_mirror(opm_angle), O2, QWP, LP2]
elements_perf_s_fun = lambda opm_angle: [O1, TL1, TL2, LP2, QWP, O2, perf_mirror(opm_angle), O2, QWP, LP]

#elements_fun = lambda opm_angle: [O1, TL1, TL2, LP, QWP, O2, O2, QWP, LP2]
#elements_s_fun = lambda opm_angle: [O1, TL1, TL2, LP2, QWP, O2, O2, QWP, LP]

s_pol = lambda opm_angle: (0, (90-opm_angle)*np.pi/180)
p_pol = (np.pi/2,0)

plot_options_p = plot_options; plot_options_s = plot_options; plot_options_un = plot_options;
plot_options_p['scale_range'] = [0,3,4]
plot_options_s['scale_range'] = [0,3,4]
plot_options_un['scale_range'] = [0,7,5]

In [42]:
# with open(save_details, 'w') as f:
# #    fs=[el.focal_length for el in elements_fun(0)]
#     f.write(",".join(str(item) for item in fs if isinstance(item, optical_elements.SineLens)))

with open(save_filename, 'a') as f:
    film_mats = [film_mat, "perfect"] 
    elements_i = [elements_fun, elements_perf_fun]
    elements_s_i = [elements_s_fun, elements_perf_s_fun]
        
    f.write("angle, film, film_thickess, polarisation, NA1, NA2, RCE, CE, EE\n")
    for i in range(2):  # loop over film and perfect
        film_mat_t = film_mats[i]
        for opm_angle in opm_angles:
            elements = elements_i[i](opm_angle)
            elements_s = elements_s_i[i](opm_angle)
            element_fs = []

            # un/depolarised
            source = dipole_source.DipoleSource()
            source.generate_dipoles(dipole_count)

            system = OpticalSystem("Dual view system (%d degrees) rapid tumbling" % opm_angle, elements, source, options)
            system.trace()
            plot_options['title'] = \
                r'Intensity in exit pupil for dOPM (%s mirror): $\theta_{OPM}=%d^{\circ}$, rapidly tumbling fluorophores' \
                % (film_mat_t, opm_angle)
            pupil_fig = system.detector.plot_pupil(**plot_options_un)
            pupil_fig.savefig(os.path.join(savepath_full, "pupil_plot_tumbling_%s_%d.png" % (film_mat_t, opm_angle)), bbox_inches='tight' )

            f.write("%0.1f, %s, %.2E, %s, %.2f, %.2f, %0.5f, %0.5f, %0.5f\n" % 
                (opm_angle, film_mat_t, film_thickness, "unpolarised", NA_O1, NA_O2,
                 system.detector.relative_collection_efficiency,
                 system.detector.emission_efficiency, 
                 system.detector.collection_efficiency)
                )

            plot_options['title'] = \
                r'Intensity incident on O1 pupil for dOPM (%s mirror): $\theta_{OPM}=%d^{\circ}$, rapidly tumbling fluorophores'\
                % (film_mat_t, opm_angle)
            pupil_fig_i = system.initial_detector.plot_pupil(**plot_options_un)
            pupil_fig_i.savefig(os.path.join(savepath_full, "initial_pupil_plot_tumbling_%s_%d.png" % (film_mat_t, opm_angle)), bbox_inches='tight' )

            # p-polarised 
            source = dipole_source.DipoleSource()
            source.generate_dipoles(dipole_count)
            source.classical_photoselection(p_pol)

            system = OpticalSystem("Dual view system (%d degrees) p-polarised" % opm_angle, elements, source, options)
            system.trace()
            plot_options['title'] = \
                r'Intensity in exit pupil for dOPM (%s mirror): $\theta_{OPM}=%d^{\circ}$, p-polarised excitation, static fluorophores'\
                % (film_mat_t, opm_angle)
            pupil_fig = system.detector.plot_pupil(**plot_options_p)
            pupil_fig.savefig(os.path.join(savepath_full, "pupil_plot_p-polarised_%s_%d.png" % (film_mat_t, opm_angle)), bbox_inches='tight' )

            f.write("%0.1f, %s, %.2E, %s, %.2f, %.2f, %0.5f, %0.5f, %0.5f\n" % 
                (opm_angle, film_mat_t, film_thickness, "p-polarised", NA_O1, NA_O2,
                 system.detector.relative_collection_efficiency,
                 system.detector.emission_efficiency, 
                 system.detector.collection_efficiency)
                )

            plot_options['title'] = \
                r'Intensity incident on O1 pupil for dOPM (%s mirror): $\theta_{OPM}=%d^{\circ}$, p-polarised excitation, static fluorophores'\
                % (film_mat_t, opm_angle)
            pupil_fig_i = system.initial_detector.plot_pupil(**plot_options_p)
            pupil_fig_i.savefig(os.path.join(savepath_full, "initial_pupil_plot_p-polarised_%s_%d.png" % (film_mat_t, opm_angle)), bbox_inches='tight')

            # s-polarised
            source = dipole_source.DipoleSource()
            source.generate_dipoles(dipole_count)
            source.classical_photoselection(s_pol(opm_angle))

            system = OpticalSystem("Dual view system (%d degrees) s-polarised" % opm_angle, elements_s, source, options)
            system.trace()
            plot_options['title'] = \
                r'Intensity in exit pupil for dOPM (%s mirror): $\theta_{OPM}=%d^{\circ}$, s-polarised excitation, static fluorophores'\
                % (film_mat_t, opm_angle)
            pupil_fig = system.detector.plot_pupil(**plot_options_s)
            pupil_fig.savefig(os.path.join(savepath_full, "pupil_plot_s-polarised_%s_%d.png" % (film_mat_t, opm_angle)), bbox_inches='tight' )

            f.write("%0.1f, %s, %.2E, %s, %.2f, %.2f, %0.5f, %0.5f, %0.5f\n" % 
                (opm_angle, film_mat_t, film_thickness, "s-polarised", NA_O1, NA_O2,
                 system.detector.relative_collection_efficiency,
                 system.detector.emission_efficiency, 
                 system.detector.collection_efficiency)
                )

            plot_options['title'] = \
                r'Intensity incident on O1 pupil for dOPM (%s mirror): $\theta_{OPM}=%d^{\circ}$, s-polarised excitation, static fluorophores'\
                % (film_mat_t, opm_angle)
            pupil_fig_i = system.initial_detector.plot_pupil(**plot_options_s)
            pupil_fig_i.savefig(os.path.join(savepath_full, "initial_pupil_plot_s-polarised_%s_%d.png" % (film_mat_t, opm_angle)), bbox_inches='tight' )



AttributeError: 'LinearPolariser' object has no attribute 'focal_length'